In [2]:
import sys, os
import numpy as np
import pandas as pd
import allel
import zarr
import numcodecs
import warnings

%matplotlib inline
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('retina', 'png')
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import seaborn as sns
sns.set()
sns.set_style("white")
sns.set_context("notebook")

## Read in meta data

In [2]:
meta_data = pd.read_excel('../data/Papio-Genomes_JR_120720_MR-CR-KM_geoloc.xlsx')
meta_data.head()

,Unnamed: 0,PGDP_ID,Provider_ID,Provider,Genus,Species,Origin,Sex,address,longitude,latitude
0,0,PD_0067,1043,Roos,Theropithecus,gelada,captive,M,"SDSU Captive Wildlife Research Facility, Brook...",-96.793280,44.334031
1,1,PD_0199,09SNF1101115,Knauf/Chuma/Roos,Papio,anubis,"Serengeti, Tanzania",F,"Serengeti, Mara, Lake Zone, Tanzania",34.742544,-1.996626
2,2,PD_0200,11SNF1101115,Knauf/Chuma/Roos,Papio,anubis,"Serengeti, Tanzania",F,"Serengeti, Mara, Lake Zone, Tanzania",34.742544,-1.996626
3,3,PD_0201,19SNM1131115,Knauf/Chuma/Roos,Papio,anubis,"Serengeti, Tanzania",M,"Serengeti, Mara, Lake Zone, Tanzania",34.742544,-1.996626
4,4,PD_0202,20SNF1131115,Knauf/Chuma/Roos,Papio,anubis,"Serengeti, Tanzania",F,"Serengeti, Mara, Lake Zone, Tanzania",34.742544,-1.996626


## Merge VCF files

Make sure all VCF files are tabix indexed. If not here is how:

    #!/bin/sh

    for VCFFILE in PD*.snps.vcf.gz ; do
        tabix -f -p vcf $VCFFILE
    done
    
Run with sbatch:

    conda activate baboons
    sbatch --mem 15000 -t 10:00:00 -J merge --chdir `pwd` ../variants_zarr/merge_vcfs.sh

Get baboon sample IDs:

In [3]:
baboon_samples = [x for x in meta_data.PGDP_ID if x.startswith('PD')] # to not get the SciAdvPaper samples
baboon_samples[:5]

['PD_0067', 'PD_0199', 'PD_0200', 'PD_0201', 'PD_0202']

Construct list of VCF file names to paste into merge_vcf.py script:

In [ ]:
suffix = '.variable.filtered.HF.snps.vcf.gz'
# get only files that exist.....
vcf_files = [x+suffix for x in baboon_samples if os.path.exists(f'../../../data/variants/{x}{suffix}')]

In [10]:
' '.join(vcf_files)

Shell script for sbatch:

**NB:** We assume missing positions are ref. That means that we must use a "samples x variants" callability mask on the resulting genotypes when extracting data using scikit-allel. That is probably a one-off operation that we can write back to the zarr data structure.

    # Merge the VCF files (and compress the output):
    bcftools merge --missing-to-ref -Oz -o ../variants_zarr/merged.variable.filtered.HF.snps.vcf.gz PD_0067.variable.filtered.HF.snps.vcf.gz PD_0199.variable.filtered.HF.snps.vcf.gz PD_0200.variable.filtered.HF.snps.vcf.gz PD_0201.variable.filtered.HF.snps.vcf.gz PD_0202.variable.filtered.HF.snps.vcf.gz.........
    
    # Make a tabix index for the merged file:
    tabix -f -p vcf merged.variable.filtered.HF.snps.vcf.gz

Run like this (in /home/kmt/primatediversity/data/variants):
    
    conda activate baboons
    sbatch --mem 15000 -t 10:00:00 -J merge --chdir `pwd` ../variants_zarr/merge_vcfs.sh

## Build zarr data structure

Extract chromosome names from one of the baboon vcf files:

In [52]:
chromosomes = !zcat ../../../data/variants/PD_0216.variable.filtered.HF.snps.vcf.gz | grep -v '#' | cut -f 1 | uniq 
chromosomes[:5]

['chr1',
 'chr10',
 'chr10_NW_021160241v1_random',
 'chr10_NW_021160242v1_random',
 'chr10_NW_021160243v1_random']

In [29]:
vcf_path = '../../../data/variants_zarr/test.vcf.gz'
zarr_path = '../../../data/variants_zarr/test.zarr'

In [16]:
!ls -lh {vcf_path}

-rw-rw-r-- 1 kmt primatediversity 1,7G 17 dec 20:30 ../../../data/variants_zarr/test.vcf.gz


In [14]:
for chrom in chromosomes:
    allel.vcf_to_zarr(vcf_path, zarr_path, 
                      group=chrom, region=chrom, fields='*',
                      # log=sys.stdout
                     )       

In [30]:
callset = zarr.open_group(zarr_path, mode='r')
callset.tree(expand=False)

Tree(nodes=(Node(disabled=True, name='/', nodes=(Node(disabled=True, name='chr1', nodes=(Node(disabled=True, n…

## Correct uncalled positions

When we merge VCF files, we treat missing positions in some individuals as REF/REF although they could also be uncalled positions. So we need to check if any positions in the merged VCF file are not in the callability mask - and if so, change them to ./..

Read callability mask for each individual:

In [29]:
import zarr
z = zarr.zeros((1, 10))
print(z.shape)
z.set_coordinate_selection(([0, 0], [1, 3]), 99)
z[:]


(1, 10)


array([[ 0., 99.,  0., 99.,  0.,  0.,  0.,  0.,  0.,  0.]])

In [37]:

z = zarr.array([[[ 0,  0],
        [ 0,  1],
        [ 1,  1]],

       [[ 0,  0],
        [ 0,  1],
        [ 0,  0]],

       [[ 0,  2],
        [ 1,  2],
        [ 2,  2]],

       [[ 0,  0],
        [ 0,  0],
        [ 1,  1]],

       [[ 0,  1],
        [ 0,  2],
        [ 1,  1]]])
z.set_orthogonal_selection(([1, 3], [1], slice(None)), -1)
z[:]

array([[[ 0,  0],
        [ 0,  1],
        [ 1,  1]],

       [[ 0,  0],
        [-1, -1],
        [ 0,  0]],

       [[ 0,  2],
        [ 1,  2],
        [ 2,  2]],

       [[ 0,  0],
        [-1, -1],
        [ 1,  1]],

       [[ 0,  1],
        [ 0,  2],
        [ 1,  1]]])

In [27]:
from collections import defaultdict

callability_dir_path = Path('../../../data/callability')
callabiility_zarr_path = '../../../data/variants_zarr/callability.zarr'

callabilit_masks = zarr.open_group(callabiility_zarr_path, mode='w')

for bed_file in callability_dir_path.glob(*.bed.gz):
    sample = bed_file.name.split('.')[0]
    coordinates = defaultdict(list)
    with open(bed_file) as f:
        for line in f:
            chrom, start, end = line.split()
            coordinates[chrom].append((int(start), int(end)))

    indiv = callabilit_masks.create_group(sample)
    for chrom in sorted(coordinates):            
        indiv.array(chrom, coordinates[chrom], dtype='int32')

callabilit_masks.tree(expand=False)

Tree(nodes=(Node(disabled=True, name='/', nodes=(Node(disabled=True, name='indiv1', nodes=(Node(disabled=True,…

In [28]:
callabilit_masks['indiv1/chr1'][:]

array([[ 2,  4],
       [ 7, 10],
       [13, 16]], dtype=int32)

In [32]:
!ls 

baboon_ranges.ipynb  callability.zarr	 Untitled.ipynb
callability.ipynb    scikit-allel.ipynb  vcf_files.txt


In [8]:


# make a zarr structure for callability masks for indiv and chrom

callability_mask = zarr.

# def get_callability_mask(indiv, chrom):
#     # read call mask from bed file
#     indiv_call_mask = [(2, 4), (7, 10), (13, 16)] # dummy
#     starts, ends = zip(*indiv_call_mask)
#     return starts, ends

# open zarr group for reading
with zarr.open_group(zarr_path, mode='r+') as callset:
    
    for callset_chrom in callset.groups():
        # get genotype array for individual and chromosome

        for sample_idx, sample in enumerate(callset_chrom['samples'][:]):

            pos = allel.SortedIndex(callset_chrom['variants/POS'])
    
            # starts and ends of called intervals
            starts, ends = zip(*callability_masks[f'{sample}/{chrom}'])

            # find snp positions that overlap callabilty mask:
            in_call_mask = np.digitize(pos, starts) == np.digitize(pos, ends) + 1

            # get index of the ones not in the callabilty mask:
            snps_not_called = np.nonzero(~in_call_mask)

            # change the missing ones from REF/REF to ./. (-1, -1):
            gt = callset_chrom['calldata/GT']
            gt.set_orthogonal_selection((snps_not_called, [sample_idx], slice(None)), -1)


(array([ 0,  1,  4,  5,  6, 10, 11, 12, 16, 17, 18, 19]),)

## Playing with scikit-allel

In [20]:
gt_zarr = callset['chr1/calldata/GT']
gt_zarr.info

Name,/chr1/calldata/GT
Type,zarr.core.Array
Data type,int8
Shape,"(3560099, 2, 2)"
Chunk shape,"(65536, 2, 2)"
Order,C
Read-only,True
Compressor,"Blosc(cname='lz4', clevel=5, shuffle=SHUFFLE, blocksize=0)"
Store type,zarr.storage.DirectoryStore
No. bytes,14240396 (13.6M)
No. bytes stored,4545191 (4.3M)


In [22]:
pos = allel.SortedIndex(callset['chr1/variants/POS'])
pos

0,1,2,3,4,...,3560094,3560095,3560096,3560097,3560098
5358,5362,5388,5447,5542,...,223615954,223615964,223616040,223616053,223616073


In [23]:
loc_region = pos.locate_range(20000000, 20100000)
loc_region

slice(375626, 377378, None)

In [24]:
gt_region = allel.GenotypeArray(gt_zarr[loc_region])
gt_region

<GenotypeArray shape=(1752, 2, 2) dtype=int8>
1/1 1/1
1/1 ./.
1/1 1/1
...
1/1 1/1
1/1 1/1
1/1 ./.

In [29]:
multi_allelic = callset['chr1/variants/numalt'][:] > 1
multi_allelic

array([False, False, False, ..., False, False, False])

In [30]:
loc_variant_selection = ~multi_allelic #& (afr_af[:, 0] > 0.05)
loc_variant_selection

array([ True,  True,  True, ...,  True,  True,  True])

In [31]:
np.count_nonzero(loc_variant_selection)


3556332

In [32]:
gt = allel.GenotypeArray(gt_zarr)
gt

<GenotypeArray shape=(3560099, 2, 2) dtype=int8>
0/1 ./.
1/1 ./.
0/1 ./.
...
1/1 ./.
1/1 ./.
1/1 ./.

In [33]:
gt_variant_selection = gt.compress(loc_variant_selection, axis=0)
gt_variant_selection

<GenotypeArray shape=(3556332, 2, 2) dtype=int8>
0/1 ./.
1/1 ./.
0/1 ./.
...
1/1 ./.
1/1 ./.
1/1 ./.

In [34]:
gt_dask = allel.GenotypeDaskArray(gt_zarr)
gt_dask

<GenotypeDaskArray shape=(3560099, 2, 2) dtype=int8>

In [35]:
gt_variant_selection = gt_dask.compress(loc_variant_selection, axis=0).compute()
gt_variant_selection

<GenotypeArray shape=(3556332, 2, 2) dtype=int8>
0/1 ./.
1/1 ./.
0/1 ./.
...
1/1 ./.
1/1 ./.
1/1 ./.

In [36]:
samples = callset['chr1/samples'][:]
samples

array(['PD_0216', 'PD_0219'], dtype=object)

In [49]:
panel = pd.DataFrame(dict(sample=['PD_0219', 'PD_0216'], super_pop=['AFR', 'EUR']))

In [50]:
samples_list = list(samples)
samples_callset_index = [samples_list.index(s) for s in panel['sample']]
panel['callset_index'] = samples_callset_index
panel.head()

,sample,super_pop,callset_index
0,PD_0219,AFR,1
1,PD_0216,EUR,0


In [51]:
loc_samples_afr = panel[panel.super_pop == 'AFR'].callset_index.values
loc_samples_afr

array([1])

In [53]:
loc_samples_afr = panel[panel.super_pop == 'AFR'].callset_index.values
loc_samples_afr

array([1])

In [54]:
gt_afr = gt_variant_selection.take(loc_samples_afr, axis=1)
gt_afr

,0
0,./.
1,./.
2,./.
...,...
3556329,./.
3556330,./.
3556331,./.


In [55]:
gt_afr = gt_dask.subset(loc_variant_selection, loc_samples_afr).compute()
gt_afr

,0
0,./.
1,./.
2,./.
...,...
3556329,./.
3556330,./.
3556331,./.
